In [1]:
%%capture

!pip install ipython-autotime
%load_ext autotime

time: 2.54 ms (started: 2023-07-27 06:58:38 +00:00)


In [2]:
%%capture
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq huggingface-hub==0.16.4 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off

time: 1min 16s (started: 2023-07-27 06:58:41 +00:00)


In [3]:
%%capture
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

time: 49.3 s (started: 2023-07-27 07:00:17 +00:00)


In [6]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

time: 437 µs (started: 2023-07-27 07:01:31 +00:00)


In [4]:

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

time: 14.2 s (started: 2023-07-27 07:01:06 +00:00)


In [5]:
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
notebook_login()

time: 76.2 ms (started: 2023-07-27 07:01:21 +00:00)


## Dataset

In [7]:
# importing the zipfile module
from zipfile import ZipFile

zip_path = "/content/archive.zip"

# loading the temp.zip and creating a zip object
with ZipFile(zip_path, 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall()

time: 350 ms (started: 2023-07-27 07:01:37 +00:00)


In [27]:
import json

file_path = '/content/train-v1.1.json'
with open(file_path, 'r') as json_file:
        data = json.load(json_file)

time: 697 ms (started: 2023-07-27 07:15:14 +00:00)


In [9]:
data['data'][0]['paragraphs'][1]['qas']

[{'answers': [{'answer_start': 248, 'text': 'September 1876'}],
  'question': 'When did the Scholastic Magazine of Notre dame begin publishing?',
  'id': '5733bf84d058e614000b61be'},
 {'answers': [{'answer_start': 441, 'text': 'twice'}],
  'question': "How often is Notre Dame's the Juggler published?",
  'id': '5733bf84d058e614000b61bf'},
 {'answers': [{'answer_start': 598, 'text': 'The Observer'}],
  'question': 'What is the daily student paper at Notre Dame called?',
  'id': '5733bf84d058e614000b61c0'},
 {'answers': [{'answer_start': 126, 'text': 'three'}],
  'question': 'How many student news papers are found at Notre Dame?',
  'id': '5733bf84d058e614000b61bd'},
 {'answers': [{'answer_start': 908, 'text': '1987'}],
  'question': 'In what year did the student paper Common Sense begin publication at Notre Dame?',
  'id': '5733bf84d058e614000b61c1'}]

time: 4.01 ms (started: 2023-07-27 07:01:52 +00:00)


In [28]:
#general function for multiple files
import os
qa_list = []
for i in data['data'][0]['paragraphs']:
  for j in i['qas']:
    answer = j['answers'][0]['text']
    question = j['question']

    qa_list.append({"question":question, "answer": answer})

data = {"questions":qa_list}



time: 1.32 ms (started: 2023-07-27 07:15:18 +00:00)


In [11]:
import json
with open("squad_dataset.json","w") as f:
  json.dump(data['questions'],f)

time: 3.27 ms (started: 2023-07-27 07:02:09 +00:00)


In [12]:
%%capture
!pip install datasets

time: 8.56 s (started: 2023-07-27 07:02:18 +00:00)


In [30]:
from datasets import load_dataset
import json
data = load_dataset("json", data_files = "squad_dataset.json",  split="train")
data

Dataset({
    features: ['question', 'answer'],
    num_rows: 269
})

time: 279 ms (started: 2023-07-27 07:16:53 +00:00)


In [32]:
from datasets import Dataset

data_with_text = Dataset.from_dict(
    {
        'text': [f"{q} {a}" for q, a in zip(data['question'], data['answer'])],
        'question': data['question'],
        'answer': data['answer'],
    }
)

time: 8.91 ms (started: 2023-07-27 07:17:19 +00:00)


## Loading the Model

In [14]:
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

time: 445 µs (started: 2023-07-27 07:02:28 +00:00)


In [15]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = (0)
tokenizer.padding_side = "left"

time: 600 ms (started: 2023-07-27 07:02:31 +00:00)


In [16]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

time: 3min 45s (started: 2023-07-27 07:02:35 +00:00)


## Finetuning

In [17]:
def generate_prompt(data_point):
  return f"""
  <human>: {data_point['question']}
  <assistant>: {data_point['answer']}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding = True, truncation = True)
  return tokenized_full_prompt

data = data.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


time: 390 ms (started: 2023-07-27 07:06:21 +00:00)


In [18]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 1
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

time: 1.08 ms (started: 2023-07-27 07:06:21 +00:00)


In [19]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=4, #LORA_R,
    lora_alpha=16, #LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=0.05, #LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 2,097,152 || all params: 6,740,512,768 || trainable%: 0.03111264783824826
time: 10.2 s (started: 2023-07-27 07:06:21 +00:00)


In [20]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.9,
  "top_p": 0.6,
  "transformers_version": "4.32.0.dev0"
}

time: 63.7 ms (started: 2023-07-27 07:06:31 +00:00)


In [21]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

time: 34.5 ms (started: 2023-07-27 07:09:30 +00:00)


In [ ]:
!pip install trl

In [33]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=data_with_text,
    peft_config=config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

time: 204 ms (started: 2023-07-27 07:17:31 +00:00)


In [34]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

time: 4.66 ms (started: 2023-07-27 07:17:56 +00:00)


In [35]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,3.285500
20,2.782200
30,2.274300
40,2.010000
50,1.960000
60,1.766300
70,1.677500
80,1.373100
90,1.248900
100,1.158600


TrainOutput(global_step=100, training_loss=1.953630075454712, metrics={'train_runtime': 682.5192, 'train_samples_per_second': 2.344, 'train_steps_per_second': 0.147, 'total_flos': 1434539462049792.0, 'train_loss': 1.953630075454712, 'epoch': 5.88})

time: 11min 23s (started: 2023-07-27 07:18:04 +00:00)


In [36]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

time: 50.4 ms (started: 2023-07-27 07:29:27 +00:00)


# Testing the fine tuned model

In [37]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

time: 75.3 ms (started: 2023-07-27 07:30:02 +00:00)


In [38]:
device = "cuda:0"
text = "What is the Grotto at Notre Dame?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


What is the Grotto at Notre Dame?
The Grotto at Notre Dame is a beautiful and peaceful place on the campus of Notre Dame University. It is a place where students, faculty, and staff can come to pray, reflect, and relax. The Grot
time: 26.6 s (started: 2023-07-27 07:30:10 +00:00)


In [39]:
text = "Where did Notre Dame students and the KKK have their encounter?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Where did Notre Dame students and the KKK have their encounter?
The KKK was a major force in the South, and the South was the home of Notre Dame.
The KKK was a major force in the South, and the South was the home of Notre Dame. (AP
time: 21.1 s (started: 2023-07-27 07:31:39 +00:00)
